# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Hoài Mẫn
- MSSV: 20127561
- Lớp: 20CLC05

# Import

In [1]:
import pandas as pd
import numpy as np
# Import thêm dữ thư viện nếu cần
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1) 
    


# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [4]:
# Phần code cho yêu cầu 1a
lr = OLSLinearRegression().fit(X_train, y_train)
lr.get_params()

array([ 1.51013627e-02,  9.02199807e-02,  4.29218175e-02,  1.39289117e-01,
       -5.67332827e-01, -1.00765115e-04,  7.40713438e-01,  1.90935798e-01,
        2.45059736e+01,  2.39351661e+00])

In [5]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_test_preds = lr.predict(X_test)
rmse = mean_squared_error(y_test, y_test_preds, squared=False)
print(rmse)

7.064046430584705


Công thức hồi quy

$$\text{Life expectancy} = theta[1]x_1 + theta[2]x_2 + theta[3]x_3 + ... + theta[10]x_{10}$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [6]:
# Phần code cho yêu cầu 1b
X_train = train.iloc[:, :-1].to_numpy()
y_train = train.iloc[:, -1].to_numpy()

# Lấy danh sách các header của bộ dữ liệu.
headers = list(train.columns.values)
# Chuyển đổi bộ dữ liệu sang kiểu numpy array để dễ dàng thao tác.
data = np.array(X_train)
# Kết quả của phương pháp hồi quy tuyến tính áp dụng trên 1 bộ dữ liệu.
def Table(data, headers):
        
    x_hat_data = []
    for i in range(10):
        row = []
        row.append(headers[i])
        row.append(data[i])
        x_hat_data.append(row)
    
    return pd.DataFrame(x_hat_data, columns=['Đặc trưng', 'RMSE'])

# Tìm ra đặc trưng tốt nhất
def ChooseBest_b(X, y):
    result = []

    for i in range(X.shape[1]):
        rmse = Cross_Validation(X[:, i:i + 1], y)
        result.append(rmse)
    
    return result, np.argmin(result)
# In ra các kết quả cross-validation như yêu cầu
def Cross_Validation(X, y):
    kf = KFold(n_splits= 5)
    list = []
    rmse = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        lr = OLSLinearRegression().fit(X_train, y_train)
        y_k_val_pred = lr.predict(X_test)
        list.append(mean_squared_error( y_test, y_k_val_pred, squared=False))
        rmse += mean_squared_error( y_test, y_k_val_pred, squared=False)

    return rmse/5
result, BestIdx = ChooseBest_b(X_train, y_train)
Table(result, headers)

,Đặc trưng,RMSE
0,Adult Mortality,46.767300
1,BMI,27.963793
2,Polio,17.912636
3,Diphtheria,16.019288
4,HIV/AIDS,69.081327
5,GDP,60.450393
6,Thinness age 10-19,51.899815
7,Thinness age 5-9,51.775059
8,Income composition of resources,13.299791
9,Schooling,11.820071


In [7]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
lr = OLSLinearRegression().fit(X_train[:, BestIdx].reshape(-1, 1), y_train)
lr.get_params()

array([5.5573994])

In [8]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
mean_squared_error(y_test, lr.predict(X_test[:, BestIdx].reshape(-1, 1)), squared=False)

10.26095039165537

Công thức hồi quy

$$\text{Life expectancy} = theta[10]*x_{10}$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [9]:
# Phần code cho yêu cầu 1c
def Table_c(data):
    x_hat_data = []
    headers = []
    headers.append('Sử dụng một đặt trưng Schooling')
    headers.append('Sử dụng hai đặt trưng có biến đổi Schooling và Income composition of resources')
    headers.append('Sử dụng hai đặt trưng có biến đổi Schooling(Schooling và Income composition of resources)')
    
    for i in range(3):
        row = []
        row.append(headers[i])
        row.append(data[i])
        x_hat_data.append(row)
    
    return pd.DataFrame(x_hat_data, columns=['Mô hình', 'RMSE'])
#Mô hình tuyến tính:  𝑌=𝜃0+𝜃9ln(𝑋9^2)
def preproces1(x10, y):
    X = np.hstack((np.ones((x10.shape[0], 1)), np.log(x10**2)))
    
    return X, y

#Mô hình tuyến tính:  𝑌=𝜃0+𝜃9𝑋9^3+𝜃10X10^5 .
def preproces2(x9,x10, y):
    X = np.hstack((np.ones((x9.shape[0], 1)),x9**3, x10**5))
    
    return X, y

#Mô hình tuyến tính:  𝑌=𝜃0 + 𝜃9𝑋9 + 𝜃10X10^5 .
def preproces3(x9,x10, y):
    X = np.hstack((np.ones((x9.shape[0], 1)),x9, x10**5))
    return X, y

# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
def ChooseBest_c(X, y):
    result = []
    rmse1 = Cross_Validation1(X, y)
    rmse2 = Cross_Validation2(X, y)
    rmse3 = Cross_Validation3(X, y)
    result.append(rmse1)
    result.append(rmse2)
    result.append(rmse3)
    return result, np.argmin(result)
# In ra các kết quả cross-validation như yêu cầu

def Cross_Validation3(A, b):
    kf = KFold(n_splits= 5)
    error_list = []
    rmse = 0

    for train_index, test_index in kf.split(A):
        A_train, A_test = A[train_index], A[test_index]
        b_train, b_test = b[train_index], b[test_index]
        X, y = preproces3(A_train[:, 0].reshape(-1, 1), A_train[:, 1].reshape(-1, 1), b_train.reshape(-1, 1))
        #A_train[:, 0].reshape(-1, 1) :lay het tat ca dong tren cot so 0
        l = OLSLinearRegression().fit(X, y)
        X_t, y_t = preproces3(A_test[:, 0].reshape(-1, 1), A_test[:,1].reshape(-1, 1), b_test.reshape(-1, 1))
        x = l.get_params()
        bias = l.predict(X_t)
        rmse+= mean_squared_error( y_t, bias, squared=False)

    return rmse/5

def Cross_Validation2(A, b):
    kf = KFold(n_splits= 5)
    error_list = []
    rmse = 0

    for train_index, test_index in kf.split(A):
        A_train, A_test = A[train_index], A[test_index]
        b_train, b_test = b[train_index], b[test_index]
        X, y = preproces2(A_train[:, 8].reshape(-1, 1), A_train[:, 9].reshape(-1, 1), b_train.reshape(-1, 1))
        #A_train[:, 0].reshape(-1, 1) :lay het tat ca dong tren cot so 0
        l = OLSLinearRegression().fit(X, y)
        X_t, y_t = preproces2(A_test[:, 8].reshape(-1, 1), A_test[:,9].reshape(-1, 1), b_test.reshape(-1, 1))
        x = l.get_params()
        bias = l.predict(X_t)
        rmse+= mean_squared_error( y_t, bias, squared=False)

    return rmse/5

def Cross_Validation1(A, b):
    kf = KFold(n_splits= 5)
    error_list = []
    rmse = 0
    for train_index, test_index in kf.split(A):
        A_train, A_test = A[train_index], A[test_index]
        b_train, b_test = b[train_index], b[test_index]
        X, y = preproces1(A_train[:, 9].reshape(-1, 1), b_train.reshape(-1, 1))
        l = OLSLinearRegression().fit(X, y)
        X_t, y_t = preproces1(A_test[:, 9].reshape(-1, 1), b_test.reshape(-1, 1))
        #x = l.get_params()
        bias = l.predict(X_t)
        rmse+= mean_squared_error( y_t, bias, squared=False)
    return rmse/5

a, b=ChooseBest_c(X_train, y_train)
a,b
Table_c(a)

,Mô hình,RMSE
0,Sử dụng một đặt trưng Schooling,6.197766
1,Sử dụng hai đặt trưng có biến đổi Schooling và...,5.004816
2,Sử dụng hai đặt trưng có biến đổi Schooling(Sc...,5.555097


In [11]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X, y = preproces2(X_test[:, 8].reshape(-1, 1), X_test[:, 9].reshape(-1, 1), y_test)
lr = OLSLinearRegression().fit( X, y)
lr.get_params(),mean_squared_error(y, lr.predict(X), squared=False)



(array([ 5.68663368e+01,  4.50617260e+01, -3.59447151e-06]),
 3.9085214376440356)

In [12]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
X, y = preproces2(X_train[:, 8].reshape(-1, 1), X_train[:, 9].reshape(-1, 1), y_train)
lr = OLSLinearRegression().fit( X, y)
lr.get_params()
mean_squared_error(y, lr.predict(X), squared=False)

4.926736017593664

Công thức hồi quy

$$\text{Life expectancy} = theta[0] + theta[9] * x_9^3 + theta[10]*x_{10}^5$$